In [ ]:

######################################################################################
#####################################################################################
######################################################################################
############################Code by PlanteafX#########################################
##########################Haseeb Ahmad (18i-1579)####################################
#######################Muhammad Zargham Masood (18i-0464)############################
###########################Masood Ahmad (18i-0755)##################################
####################################################################################
####################################################################################



#Import libraries of tensorflow and other
import numpy as np
from re import sub
import tensorflow as tf
from tensorflow.keras import metrics

import matplotlib.colors as mcolors

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image


import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image_dataset_from_directory

print("Done importing packages!")

In [ ]:
IMG_SIZE=(132, 132)     #Image size defines the input size i.e., size of image being given to model
BATCH_SIZE=16         #Defines the Batch Size 
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0) #Imagedatagenerator for training set
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)  #Imagedatagenerator for validation set
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)  #Imagedatagenerator for test set

In [ ]:
#pick the training set from the directory using imagedatagenerator train_datagen
#train_set in our case is 90%
train_set = train_datagen.flow_from_directory(
    './new/train', 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
#pick the validation set from the directory using imagedatagenerator val_datagen
#val_set in our case is 7%
val_set = val_datagen.flow_from_directory(
    './new/val', 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
#pick the test set from the directory using imagedatagenerator test_datagen
#test_set in our case is 3%
test_set = test_datagen.flow_from_directory(
    './new/test', 
    shuffle=False, 
    seed=42,
    color_mode="rgb", #color mode is kept rgb beacuse of the colored images
    class_mode="categorical",  #as data is in classes so it is important to pick in categorical form
    target_size=IMG_SIZE,  #defines the image size (132x132)
    batch_size=BATCH_SIZE) #defines the batch size (16)

In [ ]:

#Check the indices of test set whether they are correct or not
test_set.class_indices

In [ ]:
#downloading the model from keras with imagenet weights
base_model = tf.keras.applications.EfficientNetB5(
                                include_top=False,
                                weights='imagenet',
                                input_shape=IMG_SIZE+(3,),
                            )

In [ ]:
base_model.summary() #prints the base_model summary

In [ ]:
image_batch, label_batch = next(iter(train_set)) #get the image batch and label batch of each batch present in the train set
feature_batch = base_model(image_batch) #determining the shape of output by pretained model
print(feature_batch.shape)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() #make pooling layer
feature_batch_average = global_average_layer(feature_batch)  #determining the average shape of feature batch

print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(train_set.num_classes, activation="softmax") #make the dense layer for prediction
prediction_batch = prediction_layer(feature_batch_average) 
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(132, 132, 3)) #making the input layer
x = base_model(inputs, training=True)#adding the input layer in front of pretained model
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x) #adding the dropout layer
outputs = prediction_layer(x) #adding the prediction layer
model = tf.keras.Model(inputs, outputs)# making the model by giving input layer and output layer

In [ ]:
model.summary() #prints the summary of model

In [ ]:
#metrics that are shown  and stroed during training time
METRICS = [
      metrics.CategoricalAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
]

In [ ]:
#early stopping technique to stop the training when there is no chnage of loss in five epochs
callback = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    mode="auto"
)

In [ ]:

base_learning_rate = 0.001 #initial learning rate 

#uses adam as a optimizer with beta1 = 0.9 and beta2 = 0.999. Uses categorical cross entropy loss because the data is multiclass
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate, beta_1=0.9,
    beta_2=0.999),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=METRICS)

In [ ]:
initial_epochs = 10   #initializes initial epochs

In [ ]:
#training the model using val_set and train_set (It took almost 6 hrs to complete training on 10 epochs)

history = model.fit(train_set,
                    epochs=initial_epochs,validation_data=val_set, callbacks=[callback])

In [ ]:

#to evaluate the test_set on our trained model
model.evaluate(test_set)

In [ ]:
#Plots the graph of training accuracy and validation accuracy with respect to epochs
 
plt.plot(history.history['accuracy']) #training accuracy from history
plt.plot(history.history['val_accuracy']) #validation accuracy from history
plt.title('model accuracy') #title of graph
plt.ylabel('accuracy') #y-label
plt.xlabel('epoch') #x-label
plt.legend(['train', 'val'], loc='upper left') # defining legends
plt.show() #plotting


#Plots the graph of training loss and validation loss with respect to epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:

#make prediction on test set
pred=model.predict(test_set)

In [ ]:
#save weights in .h5 format for later use
model.save_weights("./results/ResNetTest_model_weigts.h5")
print("\nModel weights saved!")

In [ ]:

#make an array of 56x56 for confusion matrix because our total species are 65
a = np.zeros((56, 56))

In [ ]:

y_pred_labels=[] #array for every prediction of image in dataset
for k in range(len(pred)):
    y_pred_labels.append(np.argmax(pred[k])) #extracts the max from the prediction(predicted class) and stores it in array

In [ ]:
import math
number_of_examples = len(test_set.filenames) #extract filenames from test set
number_of_generator_calls = math.ceil(number_of_examples / (1.0 * BATCH_SIZE)) 
# 1.0 above is to skip integer division

test_labels = [] # true labels array of test set

for i in range(0,int(number_of_generator_calls)):
    test_labels.extend(np.array(test_set[i][1])) #extracts the lavel array from test set of every image and stores it in an array

In [ ]:
y_test_labels=[] #true labels
for k in range(len(test_labels)):
    y_test_labels.append(np.argmax(test_labels[k])) #get the max of true labels array for every image and stores it in an array

In [ ]:
for k in range(len(y_pred_labels)):
    #if (y_test_labels[k]== y_pred_labels[k]):
    a[y_test_labels[k]][y_pred_labels[k]] += 1  #appending the true labels that model predicted it right

In [ ]:
class_names= ['Alstonia_Scholaris___healthy',
 'Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Arjun___healthy',
 'Basil___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Chinar___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Gauva___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Jamun___healthy',
 'Jatropha___healthy',
 'Lemon___Citrus_Leaf_Miner',
 'Lemon___healthy',
 'Mango___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Pomegranate___healthy',
 'Pongami_Pinnata___healthy',
 'Raspberry___healthy',
 'Rice___BrownSpot',
 'Rice___Healthy',
 'Rice___Hispa',
 'Rice___LeafBlast',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy',
 'wheat___Healthy',
 'wheat___stripe_rust']

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#ploting the confusing matrix
df_cm = pd.DataFrame(a, index = [i for i in class_names],
                  columns = [i for i in class_names])
plt.figure(figsize = (50,20)) #showing the confusing matrix
sn.heatmap(df_cm, annot=True, fmt = 'g') 